In [1]:
import erf_plotting as et
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

components = {'p1': {'tmin': 0.08, 'tmax': 0.120},
            'n1': {'tmin': 0.130, 'tmax': 0.160},
            'p3': {'tmin': 0.300, 'tmax': 0.400},
            'n3': {'tmin': 0.500, 'tmax': 0.600},
            'p5': {'tmin': 0.700, 'tmax': 0.800},
            'n5': {'tmin': 0.900, 'tmax': 1.000}}

In [ ]:
Xbin, ybin, sessioninds = et.load_data()
Xbin[0].shape

### T-test on the variability of the difference between animate vs inanimate

In [ ]:
def conf_sesh_bin_list(X, y, sessioninds):
    across_sesh = [[], [], [],[],[],[],[]] # 7 sessions in each list, first value in each list is the first bin
    within_sesh = [[], [], [],[],[],[],[]] # 7 bins in each list, first value in each list is the first session
    
    for a in range(len(X)): # loop over bins
        X_tmp = X[a]
        y_tmp = y[a]
        for b in range(len(X)): # loop over sessions
            inds = np.where(sessioninds[a] == b)
            sXtmp = X_tmp[:, inds, :].squeeze()
            sytmp = y_tmp[inds]

            animate_inds = (sytmp == 1)
            animate_X = sXtmp[:, animate_inds, :]
            inanimate_X = sXtmp[:, ~animate_inds, :]

            # average across trials and sensors
            animate_mean = np.mean(animate_X, axis = (1, 2))
            inanimate_mean = np.mean(inanimate_X, axis = (1, 2))

            across_sesh[a].append(animate_mean-inanimate_mean)
            within_sesh[b].append(animate_mean-inanimate_mean)
    
    return across_sesh, within_sesh

def std_list(list_of_means):
    stds = []
    for i in range(len(list_of_means)):
        if i !=4:
            stds.append(np.std(np.array(list_of_means[i]), axis=0))
            
    return stds

def get_component_std(conf, tmin, tmax, sfreq):
    stds = []
    tmin = int(tmin*sfreq)
    tmax = int(tmax*sfreq)
    for i in range(len(conf)):
        std_tmp = np.mean(conf[i][tmin:tmax], axis=0)
        stds.append(std_tmp)

    return stds

In [ ]:

Xbin, ybin, sessioninds = et.load_data()

across, within = conf_sesh_bin_list(Xbin, ybin, sessioninds)
across_conf = std_list(across)
within_conf = std_list(within)
    
for comp in components.keys():
    conf_across = get_component_std(across_conf, components[comp]['tmin'], components[comp]['tmax'], 250)
    conf_within = get_component_std(within_conf, components[comp]['tmin'], components[comp]['tmax'], 250)

    t, p = stats.ttest_rel(conf_across, conf_within)
    print(f'{comp}: {pick} t = {t}, p = {p}')

    # plot the two distributions
    fig, ax = plt.subplots()
    ax.hist(conf_across, bins=20, alpha=0.5, label='across', color='blue')
    ax.hist(conf_within, bins=20, alpha=0.5, label='within', color='red')

### T-test on single trial variabilty

In [ ]:
def variablity_over_trials(X, y, sessioninds):
    across_sesh = [[], [], [],[],[],[],[]] # 7 sessions in each list, first value in each list is the first bin
    within_sesh = [[], [], [],[],[],[],[]] # 7 bins in each list, first value in each list is the first session
    
    for a in range(len(X)): # loop over bins
        X_tmp = X[a]
        y_tmp = y[a]
        for b in range(len(X)): # loop over sessions
            inds = np.where(sessioninds[a] == b)
            sXtmp = X_tmp[:, inds, :].squeeze()
            #sytmp = y_tmp[inds]

            #animate_inds = (sytmp == 1)
            #animate_X = sXtmp[:, animate_inds, :]
            #inanimate_X = sXtmp[:, ~animate_inds, :]

            # average across sensors
            all_mean = np.mean(sXtmp, axis = 2)
            # standard deviation across trials
            all_std = np.std(all_mean, axis = 1)

            across_sesh[a].append(all_std)
            within_sesh[b].append(all_std)

    return across_sesh, within_sesh


def get_component_std(std, tmin, tmax, sfreq):
    stds = [[],[],[],[],[],[],[]]
    tmin = int(tmin*sfreq)
    tmax = int(tmax*sfreq)
    
    for i in range(len(std)):
        for j in range(len(std[i])):
            std_tmp = np.mean(std[i][j][tmin:tmax])
            stds[i].append(std_tmp)

    return stds

In [ ]:
for pick in ['mag', 'grad']:
    Xbin, ybin, sessioninds = et.load_data(pick)

    across, within = variablity_over_trials(Xbin, ybin, sessioninds)
    
    for comp in components.keys():
        conf_across = get_component_std(across, components[comp]['tmin'], components[comp]['tmax'], 250)
        conf_within = get_component_std(within, components[comp]['tmin'], components[comp]['tmax'], 250)
        
        for i in range(len(conf_across)):
            t, p = stats.ttest_rel(conf_across[i], conf_within[i])
            print(f'{comp}, {i+1}: {pick} t = {t}, p = {p}')

            # plot the two distributions
            fig, ax = plt.subplots()
            ax.hist(conf_across[i], bins=20, alpha=0.5, label='across', color='blue')
            ax.hist(conf_within[i], bins=20, alpha=0.5, label='within', color='red')

## NOT ANIMATE VS INANIMATE

1. Create 7 matrices where each matrix contains only data from a certain bin or session (we have 14 matrices in total)
2. In each of these matrices average over sensors, then take the standard deviation over the all the trials (So not divided into animate vs inanimate, since for us to take the difference in animate vs inanimate erps we need to average over trials at this stage. This makes us loose variability).
    -> output is a vector with the length 250
3. The mean is taken of these vectors from tmin to tmax of the component
4. T test is done on these means (seven across trials, seven within trials)



In [7]:
def get_X_within_across(X, y, sessioninds):
    X_within = [] # within session
    y_within = [] # within session
    X_across = [X[0],X[1],X[2],X[3],X[4],X[5],X[6]] # across session
    y_across = [y[0],y[1],y[2],y[3],y[4],y[5],y[6]] # across session

    for a in range(len(X)): # loop over bins
        X_tmp = np.array(X[a])
        y_tmp = np.array(y[a])
        for b in range(len(X)): # loop over sessions
            inds = np.where(sessioninds[a] == b)
            sXtmp = X_tmp[:, inds, :].squeeze()
            sytmp = y_tmp[inds]
            
            if a == 0:
                X_within.append(sXtmp)
                y_within.append(sytmp)
            else:
                X_within[b] = np.concatenate((X_within[b], sXtmp), axis=1)
                y_within[b] = np.concatenate((y_within[b], sytmp), axis=0)

    return X_within, y_within, X_across, y_across

def get_std(X, y):
    stds = []
    for i in range(len(X)):
        # average across labels
        all_mean = np.mean(X[i], axis = 2)

        # standard deviation across trials of the difference
        std = np.std(all_mean, axis = 1)
        stds.append(std)
    return stds

def get_component_std(std, tmin, tmax, sfreq):
    stds = []
    tmin = int(tmin*sfreq)
    tmax = int(tmax*sfreq)
    
    for i in range(len(std)):
        #std_tmp = std[i][tmin:tmax]
        std_tmp = np.mean(std[i][tmin:tmax])
        if i != 4:
            stds.append(std_tmp)

    return stds




# not animate inanimate

In [1]:
import sys
sys.path.append('../decoding/')
from decoding_source import prep_data

In [3]:
Xbin, ybin, Xsesh, ysesh = prep_data()
# concatenate the data within session

In [11]:
import numpy as np
Xseshs = [np.concatenate(i, axis = 2) for i in Xsesh]
Xseshs = [np.transpose(i.squeeze(), (2,0,1)) for i in Xseshs]

In [15]:
yseshs = [np.concatenate(i, axis = 0) for i in ysesh]

In [ ]:
#X_within, y_within, X_across, y_across = get_X_within_across(Xbin, ybin, sessioninds)
std_bins = get_std(Xbin, ybin)
std_session = get_std(Xsesh, ysesh)
np.save('data/std_within.npy', std_session)

fig, axs = plt.subplots(1,2, figsize=(20,10), sharey=True)
for ax in axs:
    if ax == axs[0]:
        std = std_session
        ax.set_title('Within session', fontsize=20)
        for i in range(len(std)):
            ax.plot(std[i], label=f'Session {i+1}', alpha=0.5)
        ax.plot(np.mean(np.array(std), axis = 0), color='black', linewidth=3, label='Mean')
        #ax.fill_between(np.arange(0, 250), np.mean(np.array(std), axis = 0) - np.std(np.array(std), axis = 0), np.mean(np.array(std), axis = 0) + np.std(np.array(std), axis = 0), color='black', alpha=0.2)

    else:
        std = std_bins
        ax.set_title('Across session', fontsize=20)
        for i in range(len(std)):
            ax.plot(std[i], label=f'Bin {i+1}', alpha=0.5)
        ax.plot(np.mean(np.array(std), axis = 0), color='black', linewidth=3, label='Mean')
        #ax.fill_between(np.arange(0, 250), np.mean(np.array(std), axis = 0) - np.std(np.array(std), axis = 0), np.mean(np.array(std), axis = 0) + np.std(np.array(std), axis = 0), color='black', alpha=0.2)
        
    ax.legend(loc = 'upper right')

for a in axs.flatten():
    a.set_xticks(np.arange(0, 251, step=50), [0. , 0.2, 0.4, 0.6, 0.8, 1. ])
    a.set_xlim(0, 250)

# ylim
#axs[0].set_ylim(0, 0.1)
axs[0].set_ylabel('Standard deviation across trials', fontsize=16)